In [1]:
import sif_parser
import glob, os
import napari

In [5]:
files_path = "/Volumes/Transfer/OM_data/sandbox/test/20211126/6737/Video Files"
# files_path = "/Volumes/GroupNiggli/Ruben/OM_images_examples/10h-32m-47s.sif"
os.path.exists(files_path)

False

In [8]:
file_list = []
for files in glob.glob(files_path + "/" + "*.sif"):
    file_list.append(files)
    # print(file_list)
    print(files)

/Volumes/Transfer/OM_data/sandbox/test/20211126/6737/Video Files/10h-32m-47s.sif
/Volumes/Transfer/OM_data/sandbox/test/20211126/6737/Video Files/10h-36m-42s.sif
/Volumes/Transfer/OM_data/sandbox/test/20211126/6737/Video Files/10h-39m-07s.sif
/Volumes/Transfer/OM_data/sandbox/test/20211126/6737/Video Files/10h-41m-34s.sif
/Volumes/Transfer/OM_data/sandbox/test/20211126/6737/Video Files/11h-16m-24s.sif
/Volumes/Transfer/OM_data/sandbox/test/20211126/6737/Video Files/11h-22m-46s.sif
/Volumes/Transfer/OM_data/sandbox/test/20211126/6737/Video Files/11h-25m-12s.sif
/Volumes/Transfer/OM_data/sandbox/test/20211126/6737/Video Files/11h-27m-32s.sif


In [37]:
file_list[1]

'/Volumes/Transfer/OM_data/sandbox/test/20211126/6737/Video Files/10h-36m-42s.sif'

In [ ]:
data, info = sif_parser.np_open(file_list[1])

In [ ]:
len(info)

529

In [26]:
info.popitem(0)

('ExperimentTime', 1637915564)

In [ ]:
dict(info)

In [24]:
info.popitem(0)

('SifVersion', 65567)

In [10]:
viewer = napari.Viewer()

In [38]:
viewer.open(path=file_list[1], plugin="napari-omaas", name = "image")

[<Image layer 'image' at 0x1cc564d90>]

In [39]:
image_10slides = viewer.layers['image'].data[:20, : , :  ]

In [28]:
viewer.add_image(data = image_10slides, gamma= 0.2, name = "10s", colormap="twilight_shifted")

<Image layer '10s' at 0x1cdbd7520>

In [33]:
viewer.layers['10s'].metadata = viewer.layers['image'].metadata

In [34]:
viewer.layers['10s'].metadata

{'SifVersion': 65567,
 'ExperimentTime': 1637915799,
 'DetectorTemperature': 0.0,
 'ExposureTime': 0.0047,
 'CycleTime': 0.00480163,
 'AccumulatedCycleTime': 0.00480163,
 'AccumulatedCycles': 1,
 'StackCycleTime': 0.00480163,
 'PixelReadoutTime': 1.78571e-09,
 'GainDAC': 2.0,
 'DetectorType': 'ZYLA-5.5-CL10 \n',
 'DetectorDimensions': (1950, 370),
 'OriginalFilename': b'C:\\Users\\PanoramicMapping\\Documents\\Optical Mapping\\Recordings/2021.11.26/10h-36m-42s.sif',
 'ShutterTime': (0.0, 0.0),
 'spectrograph': 'Spectrograph_None999',
 'SifCalbVersion': 65540,
 'Calibration_data': [0.0, 1.0, 0.0, 0.0],
 'FrameAxis': b'Pixel number',
 'DataType': b'Counts',
 'ImageAxis': b'Pixel number',
 'NumberOfFrames': 500,
 'NumberOfSubImages': 1,
 'TotalLength': 360750000,
 'ImageLength': 721500,
 'xbin': 1,
 'ybin': 1,
 'timestamp_of_0': 0,
 'timestamp_of_1': 4804,
 'timestamp_of_2': 9609,
 'timestamp_of_3': 14413,
 'timestamp_of_4': 19218,
 'timestamp_of_5': 24022,
 'timestamp_of_6': 28827,
 'time